In [1]:
from faker import Faker
import numpy as np
import pandas as pd
import random as rd
from datetime import date
import math

In [2]:
#Daniela
#Puntaje de un producto -> top 5 de puntuación más alta -> dataset productos

#Cantidad de ventas mensuales por producto. Proyección de venta por producto. -> precio unitario de cada producto comparar con el total MENSUAL -> top 5 dataset ventas y productos

#Ingresos en dólares -> suma de todos los total price goupby id_farmacia -> dataset ventas

#Cantidad de descargas de la app -> count de true tiene app -> dataset clientes

#Andrés

#Unidades vendidas por farmacia -> suma de todas las cantidades goupby id_farmacia -> dataset ventas

#Costos variables en dólares  -> top 10 costo variable + costo fijo  de cada farmacia -> dataset farmacias

#Fecha de caducidad de producto -> fecha de caducidad - fecha actual -> dataset productos



#dataset de clientes (300) -> id, nombre, edad, ubicación, comentarios, tiene app,

#dataset de farmacias (15) -> id, nombre, ubicación, cantidad de productos, cantidad de ventas semanales, costo fijo, costo variable

#dataset ventas (300) -> = ["id", "id_farmacia", "id_client","name_client", "product_id", "product_name", "quantity","price", "total_price","date"]

#dataset de productos (200) -> id, nombre, precio, fecha de caducidad, cantidad de ventas semanales, puntaje, comentarios



In [3]:
products= ["id", "nombre", "price", "fecha_caducidad", "puntaje"]
clients = ["id", "nombre", "ubicacion", "tiene app"]
farmacias = ["id", "nombre", "ubicacion", "cantidad de productos", "costo fijo", "costo variable"]
sales = ["id", "id_farmacia", "id_client","name_client", "product_id", "product_name", "quantity","price", "total_price","date"]


In [4]:
df_products=pd.read_csv('./datasets/walgreens_products_dataset-crawl_feeds.csv')
df_products=df_products[["uniq_id","title","price"]]
df_products.dropna(inplace=True)
range_puntuacion=(0,5)
#range_date=('2021-05-01','2021-07-01')



In [5]:
Faker.seed(0)
faker= Faker(['en-US'])
rd.seed(0)
sample_num=50

In [6]:
from datetime import date, timedelta
import datetime
from random import choices

In [7]:
def generate_date(star, end):
    
    #star = datetime.date(2022,1,20)
    #end = datetime.date(2025,12,1)
    
    time_between_dates = end - star
    days_between_dates = time_between_dates.days
    random_number_of_days = rd.randrange(days_between_dates)
    random_date = star + datetime.timedelta(days=random_number_of_days)
    random_date = random_date.strftime("%d-%m-%Y")
    return random_date

In [8]:
generate_date(datetime.date(2022,1,20), datetime.date(2025,12,1))

'18-03-2024'

In [9]:
def generate_products(quantity,df_products,range_puntuacion,products_labels):
    list_products=[]
    for i in range(quantity):
        id = i       
        sample=df_products.sample()
        #code=int(sample['code'])
        name=sample['title'].values[0]
        price = sample['price'].values[0]
        price = price.replace("$","")

        if("/" in price):
            price= price.split("/")[1].split(" ")[0]
            
        price = float(price.replace(",",""))
        fecha = generate_date(datetime.date(2023,1,20), datetime.date(2025,12,1))
        puntaje=rd.random()*rd.randint(*range_puntuacion)
        list_products.append((id,name,price,fecha,puntaje))
    return pd.DataFrame(list_products,columns=products_labels)
 
generate_products(200, df_products, range_puntuacion, products)
# products= ["id", "nombre", "price", "fecha_caducidad", "puntaje"]

,id,nombre,price,fecha_caducidad,puntaje
0,0,Rockstar Punched Fruit Punch Flavor Energy Drink,6.99,30-05-2025,0.161938
1,1,"Alaffia EveryDay Coconut Vegan Lip Balm, Pure...",2.99,11-10-2025,0.809868
2,2,PURA D'OR Hair Thinning Therapy Shampoo,24.99,22-09-2025,0.358049
3,3,Maui Moisture Thicken & Restore + Bamboo Fiber...,7.09,18-11-2025,0.139274
4,4,Boost High Protein Complete Nutritional Drink ...,17.49,02-08-2023,1.236738
...,...,...,...,...,...
195,195,Drive Medical Clever Lite LS Walker Rollator w...,92.59,01-07-2024,0.130158
196,196,AXE Body Wash Black,5.49,30-01-2025,0.670026
197,197,Medline Overbed Table,72.49,17-09-2024,3.748507
198,198,Medline Protect Plus Protective Underwear X-La...,89.99,19-11-2024,0.000000


In [10]:
# clients = ["id", "nombre", "ubicacion", "tiene app"]
def generate_clients(quantity, clients_labels):
    list_clients=[]
    for i in range(quantity):
        id= faker.ssn()
        name = faker.name()
        location = faker.address()
        tiene_app = rd.randint(0,1)
        list_clients.append((id,name,location,tiene_app))
    return pd.DataFrame(list_clients,columns=clients_labels)
    
    

In [11]:
#farmacias = ["id", "nombre", "ubicacion", "cantidad de productos", "costo fijo", "costo variable"]
def generate_pharmacys(quantity, farmacias_labels):
    list_pharmacys=[]
    for i in range(quantity):
        id= i
        name = faker.company()
        location = faker.address()
        cant_productos = rd.randint(0,1000)
        costo_fijo = rd.randint(0,5000)
        costo_variable = rd.randint(0,5000)
        list_pharmacys.append((id,name,location,cant_productos,costo_fijo,costo_variable))
    return pd.DataFrame(list_pharmacys,columns=farmacias_labels)

In [12]:
#sales = ["id", "id_farmacia", "id_client","name_client", "product_id", "product_name", "quantity","price", "total_price","date"]
def generate_sales(quantity, sales_labels,df_products, df_farmacias, df_clients):
    list_sales=[]
    for i in range(quantity):
        id= i
        id_farmacia= rd.randint(0,df_farmacias.shape[0]-1)
        #id_client= extract random data from column df_clients.id
        index_c = rd.randint(0, df_clients.shape[0]-1)
        index_p = rd.randint(0, df_products.shape[0]-1)
        id_client= df_clients.iloc[index_c].id
        name_client = df_clients.loc[df_clients['id'] == id_client, 'nombre'].iloc[0]
        sample=df_products.sample()
        product_id = df_products.iloc[index_p].id
        

        product_name = df_products.loc[df_products['id'] == product_id, 'nombre'].iloc[0]
        cantidad = rd.randint(0,30)
        price = df_products.loc[df_products['id'] == product_id, 'price'].iloc[0]
        total_price = price*cantidad
        #print(price)
        date = generate_date(datetime.date(2022,9,20), datetime.date(2023,1,1))
        list_sales.append((id,id_farmacia,id_client,name_client,product_id,product_name,cantidad,price,total_price,date))
    return pd.DataFrame(list_sales,columns=sales_labels)

In [13]:
df_product_gen=generate_products(200,df_products,range_puntuacion,products) 
df_client_gen=generate_clients(300,clients)
df_farmacia_gen=generate_pharmacys(15,farmacias)
df_sale_gen=generate_sales(500,sales,df_product_gen,df_farmacia_gen,df_client_gen)


In [14]:
root_path='./data'

In [15]:
df_product_gen.to_csv(f'{root_path}/products.csv',index=False)
df_client_gen.to_csv(f'{root_path}/clients.csv',index=False)
df_farmacia_gen.to_csv(f'{root_path}/farmacias.csv',index=False)
df_sale_gen.to_csv(f'{root_path}/sales.csv',index=False)